## Aspect Term Extraction (ATE) Training and Fine Tuning for Large Language Models on German hospital reviews using the special OB-Tagging


In [1]:
import torch
import os

import spacy
import ast  # To safely evaluate strings as Python objects

from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from datasets import Dataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import evaluate

from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

# We need the sys package to load modules from another directory:
import sys
sys.path.append('../')
from functions.ate_model_train_OB import *

/home/sc.uni-leipzig.de/ch31qoni/venv/absa/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Is CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("GPU device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

Is CUDA available: True
CUDA version: 12.6
GPU device name: NVIDIA A30


In [3]:
# Load the dataset into a DataFrame
data = pd.read_csv("./data/hospitalABSA/patient_review_labels_absa.csv")
data_ano = pd.read_csv("./data/hospitalABSA/patient_review_labels_absa_ano.csv")

In [4]:
models = ["google-bert/bert-base-german-cased","dbmdz/bert-base-german-cased", "dbmdz/bert-base-german-uncased",
          "FacebookAI/xlm-roberta-base", "TUM/GottBERT_base_best", "TUM/GottBERT_filtered_base_best", "TUM/GottBERT_base_last",
          "distilbert/distilbert-base-german-cased", "GerMedBERT/medbert-512", "deepset/gbert-base"]

### 1. Train standard ATE Models for 5, 6, 7, 8, 10, 12 epochs

In [5]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=5, save=True)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 4695.36 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_OB.py:420: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-ASPECT' 'O']
{0: 4.755042290175667, 1: 0.5587538226299694}
Training results for google-bert/bert-base-german-cased with 5 epochs and random seeds: 42, 42



...

['B-ASPECT' 'O']
{0: 4.755042290175667, 1: 0.5587538226299694}
Training results for deepset/gbert-base with 5 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.129026,0.808000,0.795276,0.801587
2,0.190300,0.153711,0.833333,0.826772,0.830040
3,0.095500,0.203675,0.836735,0.807087,0.821643
4,0.038200,0.209330,0.830769,0.850394,0.840467
5,0.023900,0.232320,0.825095,0.854331,0.839458



Best Model saved at: ./saved_models/ate_deepset_gbert-base_42_42_5

Tokenizer for best Model saved at: ./saved_tokenizers/ate_deepset_gbert-base_42_42_5
mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 4324.51 examples/s]


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.91      0.79      0.84       315

   micro avg       0.91      0.79      0.84       315
   macro avg       0.91      0.79      0.84       315
weighted avg       0.91      0.79      0.84       315

Precision Score: 0.9084249084249084
Recall Score: 0.7873015873015873
F1 Score: 0.8435374149659863
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labe

In [5]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=6, save=True)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 4640.89 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_OB.py:420: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-ASPECT' 'O']
{0: 4.755042290175667, 1: 0.5587538226299694}
Training results for google-bert/bert-base-german-cased with 6 epochs and random seeds: 42, 42



...

['B-ASPECT' 'O']
{0: 4.755042290175667, 1: 0.5587538226299694}
Training results for deepset/gbert-base with 6 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.141743,0.832618,0.763780,0.796715
2,0.195200,0.146101,0.832700,0.862205,0.847195
3,0.094100,0.255068,0.884444,0.783465,0.830898
4,0.032100,0.248563,0.843373,0.826772,0.834990
5,0.020800,0.272559,0.830769,0.850394,0.840467
6,0.020800,0.302068,0.835341,0.818898,0.827038



Best Model saved at: ./saved_models/ate_deepset_gbert-base_42_42_6

Tokenizer for best Model saved at: ./saved_tokenizers/ate_deepset_gbert-base_42_42_6
mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 4461.56 examples/s]


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.85      0.81      0.83       315

   micro avg       0.85      0.81      0.83       315
   macro avg       0.85      0.81      0.83       315
weighted avg       0.85      0.81      0.83       315

Precision Score: 0.85
Recall Score: 0.8095238095238095
F1 Score: 0.8292682926829269
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O',

In [6]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=7, save=True)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 4778.19 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_OB.py:420: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-ASPECT' 'O']
{0: 4.755042290175667, 1: 0.5587538226299694}
Training results for google-bert/bert-base-german-cased with 7 epochs and random seeds: 42, 42



...

['B-ASPECT' 'O']
{0: 4.755042290175667, 1: 0.5587538226299694}
Training results for deepset/gbert-base with 7 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.123697,0.821429,0.814961,0.818182
2,0.190100,0.145844,0.831373,0.834646,0.833006
3,0.089500,0.233380,0.860169,0.799213,0.828571
4,0.033400,0.235083,0.843373,0.826772,0.834990
5,0.020000,0.276270,0.813688,0.842520,0.827853
6,0.020000,0.298147,0.819923,0.842520,0.831068
7,0.007200,0.300986,0.831373,0.834646,0.833006



Best Model saved at: ./saved_models/ate_deepset_gbert-base_42_42_7

Tokenizer for best Model saved at: ./saved_tokenizers/ate_deepset_gbert-base_42_42_7
mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 4474.49 examples/s]


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.91      0.77      0.84       315

   micro avg       0.91      0.77      0.84       315
   macro avg       0.91      0.77      0.84       315
weighted avg       0.91      0.77      0.84       315

Precision Score: 0.9070631970260223
Recall Score: 0.7746031746031746
F1 Score: 0.8356164383561643
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labe

In [5]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=8, save=True)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 4816.16 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_OB.py:420: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-ASPECT' 'O']
{0: 4.755042290175667, 1: 0.5587538226299694}
Training results for google-bert/bert-base-german-cased with 8 epochs and random seeds: 42, 42



...

['B-ASPECT' 'O']
{0: 4.755042290175667, 1: 0.5587538226299694}
Training results for deepset/gbert-base with 8 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.142960,0.839130,0.759843,0.797521
2,0.190500,0.138249,0.861925,0.811024,0.835700
3,0.089300,0.231621,0.845188,0.795276,0.819473
4,0.031600,0.226515,0.841270,0.834646,0.837945
5,0.021000,0.283137,0.833977,0.850394,0.842105
6,0.021000,0.272008,0.833333,0.866142,0.849421
7,0.007200,0.304230,0.844000,0.830709,0.837302
8,0.004300,0.285747,0.833977,0.850394,0.842105



Best Model saved at: ./saved_models/ate_deepset_gbert-base_42_42_8

Tokenizer for best Model saved at: ./saved_tokenizers/ate_deepset_gbert-base_42_42_8
mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 4399.89 examples/s]


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.87      0.80      0.84       315

   micro avg       0.87      0.80      0.84       315
   macro avg       0.87      0.80      0.84       315
weighted avg       0.87      0.80      0.84       315

Precision Score: 0.8724137931034482
Recall Score: 0.8031746031746032
F1 Score: 0.8363636363636363
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labe

In [6]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=10, save=True)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 4784.23 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_OB.py:420: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-ASPECT' 'O']
{0: 4.755042290175667, 1: 0.5587538226299694}
Training results for google-bert/bert-base-german-cased with 10 epochs and random seeds: 42, 42



...

['B-ASPECT' 'O']
{0: 4.755042290175667, 1: 0.5587538226299694}
Training results for deepset/gbert-base with 10 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.144171,0.815261,0.799213,0.807157
2,0.190400,0.174801,0.811765,0.814961,0.813360
3,0.089000,0.237684,0.861472,0.783465,0.820619
4,0.033400,0.263219,0.860759,0.803150,0.830957
5,0.018300,0.326294,0.845833,0.799213,0.821862
6,0.018300,0.352128,0.823308,0.862205,0.842308
7,0.007900,0.395983,0.862069,0.787402,0.823045
8,0.001200,0.388542,0.830040,0.826772,0.828402
9,0.000700,0.393355,0.836735,0.807087,0.821643
10,0.001500,0.392739,0.841463,0.814961,0.828000



Best Model saved at: ./saved_models/ate_deepset_gbert-base_42_42_10

Tokenizer for best Model saved at: ./saved_tokenizers/ate_deepset_gbert-base_42_42_10
mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 4431.11 examples/s]


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.90      0.79      0.84       315

   micro avg       0.90      0.79      0.84       315
   macro avg       0.90      0.79      0.84       315
weighted avg       0.90      0.79      0.84       315

Precision Score: 0.8953068592057761
Recall Score: 0.7873015873015873
F1 Score: 0.8378378378378378
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labe

In [ ]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=12, save=True)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 4766.04 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_OB.py:420: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-ASPECT' 'O']
{0: 4.755042290175667, 1: 0.5587538226299694}
Training results for google-bert/bert-base-german-cased with 12 epochs and random seeds: 42, 42



...

Device set to use cuda:0
Some weights of BertForTokenClassification were not initialized from the model checkpoint at GerMedBERT/medbert-512 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 4597.02 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_OB.py:420: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-ASPECT' 'O']
{0: 4.755042290175667, 1: 0.5587538226299694}
Training results for GerMedBERT/medbert-512 with 12 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.110661,0.837209,0.779221,0.807175
2,0.189600,0.157961,0.829787,0.844156,0.836910


In [5]:
for model in ["GerMedBERT/medbert-512", "deepset/gbert-base"]:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=12, save=True)
    print()

training and results for GerMedBERT/medbert-512:


BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Device set to use cuda:0
Some weights of BertForTokenClassification were not initialized from the model checkpoint at GerMedBERT/medbert-512 and are newly initializ

mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 4681.61 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_OB.py:420: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for GerMedBERT/medbert-512 with 12 epochs and random seeds: 42, 42



...

['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for deepset/gbert-base with 12 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.138880,0.851528,0.767717,0.807453
2,0.187600,0.168863,0.837945,0.834646,0.836292
3,0.093600,0.234688,0.857724,0.830709,0.844000
4,0.034600,0.216671,0.828358,0.874016,0.850575
5,0.020800,0.307312,0.832700,0.862205,0.847195
6,0.020800,0.341299,0.856574,0.846457,0.851485
7,0.008900,0.335284,0.856574,0.846457,0.851485
8,0.005600,0.331235,0.844358,0.854331,0.849315
9,0.002200,0.386206,0.827068,0.866142,0.846154
10,0.002300,0.381521,0.849802,0.846457,0.848126



Best Model saved at: ./saved_models/ate_deepset_gbert-base_42_42_12

Tokenizer for best Model saved at: ./saved_tokenizers/ate_deepset_gbert-base_42_42_12
mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 4484.99 examples/s]


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.90      0.78      0.83       315

   micro avg       0.90      0.78      0.83       315
   macro avg       0.90      0.78      0.83       315
weighted avg       0.90      0.78      0.83       315

Precision Score: 0.9007352941176471
Recall Score: 0.7777777777777778
F1 Score: 0.8347529812606475
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labe

### 2. Train category-aware ATE Models for 5, 6, 7, 8, 10, 12 epochs

In [6]:
for model in models:
    print(f'training and results for {model}:')
    ate_cat_model(data, model, rn1=42, rn2=42, epochs=5, save=True)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 4847.46 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_OB.py:698: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-mediz. Service' 'B-Pflegepersonal' 'B-Arzt' 'B-anderer Service'
 'B-Krankenhaus' 'B-Personal']
{0: 0.1596439493228484, 1: 4.959959280624364, 2: 14.014381591562799, 3: 6.8239962651727355, 4: 10.546176046176047, 5: 5.983217355710193, 6: 18.317042606516292}
Training google-bert/bert-base-german-cased for 5 epochs with random seeds 42, 42



...

['O' 'B-mediz. Service' 'B-Pflegepersonal' 'B-Arzt' 'B-anderer Service'
 'B-Krankenhaus' 'B-Personal']
{0: 0.1596439493228484, 1: 4.959959280624364, 2: 14.014381591562799, 3: 6.8239962651727355, 4: 10.546176046176047, 5: 5.983217355710193, 6: 18.317042606516292}
Training deepset/gbert-base for 5 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.230418,0.686636,0.586614,0.632696
2,0.337500,0.192138,0.764151,0.637795,0.695279
3,0.147500,0.219408,0.752137,0.692913,0.721311
4,0.059800,0.244842,0.768240,0.704724,0.735113
5,0.037300,0.257871,0.759184,0.732283,0.745491



Best Model saved at: ./saved_models/ate_cat_deepset_gbert-base_42_42_5

Tokenizer for best Model saved at: ./saved_tokenizers/ate_cat_deepset_gbert-base_42_42_5
Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 4399.57 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.90      0.95      0.92        55
    Krankenhaus       0.90      0.62      0.73       117
       Personal       0.73      0.62      0.67        13
 Pflegepersonal       1.00      0.94      0.97        18
anderer Service       0.73      0.46      0.56        35
 mediz. Service       0.88      0.73      0.79        77

      micro avg       0.88      0.70      0.78       315
      macro avg       0.85      0.72      0.77       315
   weighted avg       0.87      0.70      0.77       315

Precision Score: 0.876984126984127
Recall Score: 0.7015873015873015
F1 Score: 0.7795414462081128
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O'

In [7]:
for model in models:
    print(f'training and results for {model}:')
    ate_cat_model(data, model, rn1=42, rn2=42, epochs=6, save=True)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 4646.03 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_OB.py:698: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-mediz. Service' 'B-Pflegepersonal' 'B-Arzt' 'B-anderer Service'
 'B-Krankenhaus' 'B-Personal']
{0: 0.1596439493228484, 1: 4.959959280624364, 2: 14.014381591562799, 3: 6.8239962651727355, 4: 10.546176046176047, 5: 5.983217355710193, 6: 18.317042606516292}
Training google-bert/bert-base-german-cased for 6 epochs with random seeds 42, 42



...

['O' 'B-mediz. Service' 'B-Pflegepersonal' 'B-Arzt' 'B-anderer Service'
 'B-Krankenhaus' 'B-Personal']
{0: 0.1596439493228484, 1: 4.959959280624364, 2: 14.014381591562799, 3: 6.8239962651727355, 4: 10.546176046176047, 5: 5.983217355710193, 6: 18.317042606516292}
Training deepset/gbert-base for 6 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.238325,0.688372,0.582677,0.631130
2,0.330400,0.185145,0.750000,0.661417,0.702929
3,0.142500,0.218466,0.793249,0.740157,0.765784
4,0.057400,0.230707,0.787234,0.728346,0.756646
5,0.034700,0.236863,0.765385,0.783465,0.774319
6,0.034700,0.241630,0.786561,0.783465,0.785010



Best Model saved at: ./saved_models/ate_cat_deepset_gbert-base_42_42_6

Tokenizer for best Model saved at: ./saved_tokenizers/ate_cat_deepset_gbert-base_42_42_6
Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 4378.90 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.88      0.96      0.92        55
    Krankenhaus       0.92      0.82      0.87       117
       Personal       0.64      0.69      0.67        13
 Pflegepersonal       1.00      0.94      0.97        18
anderer Service       0.59      0.46      0.52        35
 mediz. Service       0.84      0.75      0.79        77

      micro avg       0.86      0.79      0.82       315
      macro avg       0.81      0.77      0.79       315
   weighted avg       0.85      0.79      0.82       315

Precision Score: 0.8556701030927835
Recall Score: 0.7904761904761904
F1 Score: 0.8217821782178217
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O

In [8]:
for model in models:
    print(f'training and results for {model}:')
    ate_cat_model(data, model, rn1=42, rn2=42, epochs=7, save=True)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 4616.46 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_OB.py:698: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-mediz. Service' 'B-Pflegepersonal' 'B-Arzt' 'B-anderer Service'
 'B-Krankenhaus' 'B-Personal']
{0: 0.1596439493228484, 1: 4.959959280624364, 2: 14.014381591562799, 3: 6.8239962651727355, 4: 10.546176046176047, 5: 5.983217355710193, 6: 18.317042606516292}
Training google-bert/bert-base-german-cased for 7 epochs with random seeds 42, 42



...

['O' 'B-mediz. Service' 'B-Pflegepersonal' 'B-Arzt' 'B-anderer Service'
 'B-Krankenhaus' 'B-Personal']
{0: 0.1596439493228484, 1: 4.959959280624364, 2: 14.014381591562799, 3: 6.8239962651727355, 4: 10.546176046176047, 5: 5.983217355710193, 6: 18.317042606516292}
Training deepset/gbert-base for 7 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.238920,0.709845,0.539370,0.612975
2,0.327400,0.200220,0.766520,0.685039,0.723493
3,0.144800,0.232330,0.765217,0.692913,0.727273
4,0.057800,0.245363,0.750000,0.732283,0.741036
5,0.034800,0.260847,0.736059,0.779528,0.757170
6,0.034800,0.271232,0.746154,0.763780,0.754864
7,0.011000,0.281120,0.753846,0.771654,0.762646



Best Model saved at: ./saved_models/ate_cat_deepset_gbert-base_42_42_7

Tokenizer for best Model saved at: ./saved_tokenizers/ate_cat_deepset_gbert-base_42_42_7
Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 4417.75 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.88      0.93      0.90        55
    Krankenhaus       0.90      0.72      0.80       117
       Personal       0.75      0.69      0.72        13
 Pflegepersonal       0.85      0.94      0.89        18
anderer Service       0.81      0.49      0.61        35
 mediz. Service       0.76      0.78      0.77        77

      micro avg       0.84      0.76      0.80       315
      macro avg       0.83      0.76      0.78       315
   weighted avg       0.84      0.76      0.79       315

Precision Score: 0.8409893992932862
Recall Score: 0.7555555555555555
F1 Score: 0.7959866220735786
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O

In [9]:
for model in models:
    print(f'training and results for {model}:')
    ate_cat_model(data, model, rn1=42, rn2=42, epochs=8, save=True)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 4763.63 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_OB.py:698: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-mediz. Service' 'B-Pflegepersonal' 'B-Arzt' 'B-anderer Service'
 'B-Krankenhaus' 'B-Personal']
{0: 0.1596439493228484, 1: 4.959959280624364, 2: 14.014381591562799, 3: 6.8239962651727355, 4: 10.546176046176047, 5: 5.983217355710193, 6: 18.317042606516292}
Training google-bert/bert-base-german-cased for 8 epochs with random seeds 42, 42



...

['O' 'B-mediz. Service' 'B-Pflegepersonal' 'B-Arzt' 'B-anderer Service'
 'B-Krankenhaus' 'B-Personal']
{0: 0.1596439493228484, 1: 4.959959280624364, 2: 14.014381591562799, 3: 6.8239962651727355, 4: 10.546176046176047, 5: 5.983217355710193, 6: 18.317042606516292}
Training deepset/gbert-base for 8 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.227274,0.712329,0.614173,0.659619
2,0.316000,0.181655,0.752066,0.716535,0.733871
3,0.146400,0.209302,0.776062,0.791339,0.783626
4,0.057600,0.272403,0.762097,0.744094,0.752988
5,0.036100,0.265152,0.736059,0.779528,0.757170
6,0.036100,0.258910,0.785992,0.795276,0.790607
7,0.013400,0.280936,0.754579,0.811024,0.781784
8,0.006000,0.283672,0.760300,0.799213,0.779271



Best Model saved at: ./saved_models/ate_cat_deepset_gbert-base_42_42_8

Tokenizer for best Model saved at: ./saved_tokenizers/ate_cat_deepset_gbert-base_42_42_8
Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 4442.06 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.93      0.93      0.93        55
    Krankenhaus       0.90      0.65      0.76       117
       Personal       0.70      0.54      0.61        13
 Pflegepersonal       0.90      1.00      0.95        18
anderer Service       0.71      0.57      0.63        35
 mediz. Service       0.85      0.73      0.78        77

      micro avg       0.87      0.72      0.79       315
      macro avg       0.83      0.74      0.78       315
   weighted avg       0.87      0.72      0.78       315

Precision Score: 0.8669201520912547
Recall Score: 0.7238095238095238
F1 Score: 0.7889273356401384
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O

In [5]:
for model in models:
    print(f'training and results for {model}:')
    ate_cat_model(data, model, rn1=42, rn2=42, epochs=10, save=True)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 4842.75 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_OB.py:698: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-Krankenhaus' 'B-anderer Service' 'B-Pflegepersonal' 'B-Personal'
 'B-Arzt' 'B-mediz. Service' 'O']
{0: 5.983217355710193, 1: 10.546176046176047, 2: 14.014381591562799, 3: 18.317042606516292, 4: 6.8239962651727355, 5: 4.959959280624364, 6: 0.1596439493228484}
Training google-bert/bert-base-german-cased for 10 epochs with random seeds 42, 42



...

['B-Krankenhaus' 'B-anderer Service' 'B-Pflegepersonal' 'B-Personal'
 'B-Arzt' 'B-mediz. Service' 'O']
{0: 5.983217355710193, 1: 10.546176046176047, 2: 14.014381591562799, 3: 18.317042606516292, 4: 6.8239962651727355, 5: 4.959959280624364, 6: 0.1596439493228484}
Training deepset/gbert-base for 10 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.212766,0.789216,0.633858,0.703057
2,0.324100,0.201206,0.802817,0.673228,0.732334
3,0.147400,0.207261,0.787500,0.744094,0.765182
4,0.062800,0.262745,0.814815,0.692913,0.748936
5,0.038500,0.276008,0.739300,0.748031,0.743640
6,0.038500,0.273169,0.795745,0.736220,0.764826
7,0.015600,0.308614,0.750958,0.771654,0.761165
8,0.007800,0.317918,0.772549,0.775591,0.774067
9,0.004800,0.330287,0.765625,0.771654,0.768627
10,0.001700,0.328902,0.767442,0.779528,0.773438



Best Model saved at: ./saved_models/ate_cat_deepset_gbert-base_42_42_10

Tokenizer for best Model saved at: ./saved_tokenizers/ate_cat_deepset_gbert-base_42_42_10
Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 4431.89 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.86      0.93      0.89        55
    Krankenhaus       0.92      0.72      0.81       117
       Personal       0.69      0.69      0.69        13
 Pflegepersonal       0.85      0.94      0.89        18
anderer Service       0.78      0.51      0.62        35
 mediz. Service       0.76      0.74      0.75        77

      micro avg       0.84      0.75      0.79       315
      macro avg       0.81      0.76      0.78       315
   weighted avg       0.84      0.75      0.79       315

Precision Score: 0.8398576512455516
Recall Score: 0.7492063492063492
F1 Score: 0.7919463087248322
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O

In [6]:
for model in models:
    print(f'training and results for {model}:')
    ate_cat_model(data, model, rn1=42, rn2=42, epochs=12, save=True)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 4566.35 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_OB.py:698: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-Krankenhaus' 'B-anderer Service' 'B-Pflegepersonal' 'B-Personal'
 'B-Arzt' 'B-mediz. Service' 'O']
{0: 5.983217355710193, 1: 10.546176046176047, 2: 14.014381591562799, 3: 18.317042606516292, 4: 6.8239962651727355, 5: 4.959959280624364, 6: 0.1596439493228484}
Training google-bert/bert-base-german-cased for 12 epochs with random seeds 42, 42



...

['B-Krankenhaus' 'B-anderer Service' 'B-Pflegepersonal' 'B-Personal'
 'B-Arzt' 'B-mediz. Service' 'O']
{0: 5.983217355710193, 1: 10.546176046176047, 2: 14.014381591562799, 3: 18.317042606516292, 4: 6.8239962651727355, 5: 4.959959280624364, 6: 0.1596439493228484}
Training deepset/gbert-base for 12 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.227519,0.702326,0.594488,0.643923
2,0.329600,0.194476,0.741525,0.688976,0.714286
3,0.150800,0.210085,0.790514,0.787402,0.788955
4,0.056400,0.246850,0.782609,0.779528,0.781065
5,0.032300,0.307670,0.719858,0.799213,0.757463
6,0.032300,0.264187,0.785185,0.834646,0.809160
7,0.014000,0.264680,0.784906,0.818898,0.801541
8,0.006200,0.302801,0.754647,0.799213,0.776291
9,0.003700,0.320325,0.759857,0.834646,0.795497
10,0.001900,0.301080,0.790875,0.818898,0.804642



Best Model saved at: ./saved_models/ate_cat_deepset_gbert-base_42_42_12

Tokenizer for best Model saved at: ./saved_tokenizers/ate_cat_deepset_gbert-base_42_42_12
Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 4420.30 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.87      0.95      0.90        55
    Krankenhaus       0.89      0.79      0.84       117
       Personal       0.75      0.69      0.72        13
 Pflegepersonal       0.89      0.94      0.92        18
anderer Service       0.70      0.46      0.55        35
 mediz. Service       0.82      0.79      0.81        77

      micro avg       0.85      0.79      0.82       315
      macro avg       0.82      0.77      0.79       315
   weighted avg       0.84      0.79      0.81       315

Precision Score: 0.8464163822525598
Recall Score: 0.7873015873015873
F1 Score: 0.8157894736842106
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O